In [14]:
!pip install faker

In [16]:
import csv
import random
from faker import Faker

fake = Faker()

# Define the number of records
num_records = 1000000

# Open a file to write
with open('sample_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["first_name", "last_name", "address", "date_of_birth"])

    # Write the data
    for _ in range(num_records):
        writer.writerow([fake.first_name(), fake.last_name(), fake.address(), fake.date_of_birth()])

print("CSV file created successfully.")

CSV file created successfully.


In [17]:
!pip install pyspark

In [18]:
!pip install cryptography

In [19]:
from cryptography.fernet import Fernet
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Generate a key for encryption
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Create a Spark session
spark = SparkSession.builder.appName("AnonymizeData").getOrCreate()

# Load the data
df = spark.read.format("csv").option("header", "true").option("delimiter", ",").option("quote", '"').option("escape", '"').option("multiline", "true").load("sample_data.csv")

# Define a UDF to encrypt the data
def encrypt_data(x):
    return cipher_suite.encrypt(x.encode()).decode() if x is not None else None
encrypt_udf = udf(encrypt_data, StringType())

# Anonymize the columns
df = df.withColumn('first_name', encrypt_udf(df['first_name'])) \
       .withColumn('last_name', encrypt_udf(df['last_name'])) \
       .withColumn('address', encrypt_udf(df['address']))

# Save the anonymized data
df.write.csv('anonymized_large_data.csv', header=True, mode='overwrite')

# Save the key to a file
with open('key.key', 'wb') as key_file:
    key_file.write(key)

print("data anonymized successfully.")

# Stop the Spark session

Large data anonymized successfully.


In [20]:

# from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("AnonymizeData").getOrCreate()

# df_valid = spark.read.format("csv").option("header", "true").option("delimiter", ",").option("quote", '"').option("escape", '"').option("multiline", "true").load("sample_data.csv")

# # Show the schema and first few rows to verify
# df_valid.printSchema()
# df_valid.show(truncate=False)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [21]:
df.show()

+--------------------+--------------------+--------------------+-------------+
|          first_name|           last_name|             address|date_of_birth|
+--------------------+--------------------+--------------------+-------------+
|gAAAAABmjWouR2E_f...|gAAAAABmjWouCDKLI...|gAAAAABmjWouJhTL9...|   1922-03-19|
|gAAAAABmjWouWc4zT...|gAAAAABmjWouk6jq0...|gAAAAABmjWouV8t0T...|   1972-11-15|
|gAAAAABmjWou-E7n6...|gAAAAABmjWouR2zR2...|gAAAAABmjWoudkkho...|   1948-12-09|
|gAAAAABmjWougfhz_...|gAAAAABmjWouUo9z4...|gAAAAABmjWou0Vwgd...|   1974-03-26|
|gAAAAABmjWouHUWhg...|gAAAAABmjWoumSAaf...|gAAAAABmjWou--akq...|   1985-04-20|
|gAAAAABmjWouznxPt...|gAAAAABmjWou22hUz...|gAAAAABmjWoub6ygH...|   1923-02-05|
|gAAAAABmjWouRUTnX...|gAAAAABmjWou4Lrgo...|gAAAAABmjWou490bc...|   1920-10-09|
|gAAAAABmjWou6cwwv...|gAAAAABmjWouWcWBQ...|gAAAAABmjWou768l7...|   1931-11-04|
|gAAAAABmjWoubqxhF...|gAAAAABmjWouMCeoH...|gAAAAABmjWouJw-tX...|   1922-11-04|
|gAAAAABmjWouDgcCi...|gAAAAABmjWouGhXpN...|gAAAAABmj

In [24]:
from cryptography.fernet import Fernet
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Load the key
with open('key.key', 'rb') as key_file:
    key = key_file.read()

cipher_suite = Fernet(key)

# Create a Spark session
spark = SparkSession.builder.appName("DeAnonymizeData").getOrCreate()

# Load the anonymized data
df = spark.read.csv('anonymized_large_data.csv', header=True, inferSchema=True)

# Define a UDF to decrypt the data
def decrypt_data(x):
    return cipher_suite.decrypt(x.encode()).decode() if x is not None else None
decrypt_udf = udf(decrypt_data, StringType())

# De-anonymize the columns
df = df.withColumn('first_name', decrypt_udf(df['first_name'])) \
       .withColumn('last_name', decrypt_udf(df['last_name'])) \
       .withColumn('address', decrypt_udf(df['address']))

# Save the de-anonymized data
df.write.csv('deanonymized_large_data.csv', header=True, mode='overwrite')

print("Large data de-anonymized successfully.")

# Stop the Spark session
# spark.stop()

Large data de-anonymized successfully.


In [25]:
df.show()

+----------+----------+--------------------+-------------+
|first_name| last_name|             address|date_of_birth|
+----------+----------+--------------------+-------------+
|   Matthew|    Miller|0188 Bennett Way ...|   1922-03-19|
|   Rebecca|    Romero|1080 Hall Terrace...|   1972-11-15|
|   Charles|     Velez|USS Vasquez\nFPO ...|   1948-12-09|
|     James|     Ortiz|18721 Freeman Vis...|   1974-03-26|
|   Colleen|      Ryan|50216 Anna Terrac...|   1985-04-20|
|    Robert|     Green|1422 Solis Stream...|   1923-02-05|
|    Andrew|     Smith|USNS Juarez\nFPO ...|   1920-10-09|
|  Jennifer|   Webster|9121 Cole Road Ap...|   1931-11-04|
|     Haley|     Lopez|895 Jenna Prairie...|   1922-11-04|
|      John|    Mercer|4977 Carmen Burgs...|   1949-12-15|
|     Sarah|    Garcia|3067 Jones Mills ...|   1974-09-26|
|    Edward|   Jackson|52935 Marcia Moun...|   1981-11-20|
|     Aaron|    Newman|97220 Price Key\n...|   1965-12-29|
|    Monica|     Myers|6755 Porter Freew...|   2000-02-1